In [69]:
import json
import time
from scraper import getheadlines, getheadlines_JS
from textanalyser import generatesentiment
import boto
from time import gmtime, strftime

In [70]:
def generateheadlinefile(source_file, output_file, s3key, scraperfn):
    with open(source_file) as data_file:
        sources = json.load(data_file)
        data = {}
        data["time"] = time.strftime("%c")
        data["news-data"] = []

    for source in sources:
        headlines = scraperfn(source["url"], source["xpath"])
        sentiments = [generatesentiment("headline", headline) for headline in headlines]
        print(sentiments)

        data["news-data"].append({
            "name": source["name"],
            "sentiments": sentiments
        })  
        
#     with open(output_file, 'w') as outfile:
#         json.dump(data, outfile)
    
    filename = strftime("%Y-%m-%d", gmtime())
    s3 = boto.connect_s3()
    bucket = s3.create_bucket('data.newsemote')
    key = bucket.new_key(s3key + '/' + filename + '.json')
    key.set_contents_from_string(json.dumps(data))
    key.set_acl('public-read')
    
    #keep copy of file in latest.json
    latest = bucket.new_key(s3key + '/latest.json')
    latest.set_contents_from_string(json.dumps(data))
    latest.set_acl('public-read')
    
    #retrieve files in bucket
    files = bucket.list(prefix=s3key)
    files = sorted(files, key=lambda k: k.last_modified, reverse=True)
    filtered = [f for f in files if f.name != s3key + '/latest.json' and f.name != 'last7.json']
    filtered = filtered[:7]
    filelist = [file.name for file in filtered]
    print(filelist)
    
    #write to list file
    latestlist = bucket.new_key(s3key + '/last7.json')
    latestlist.set_contents_from_string(json.dumps(filelist))
    latestlist.set_acl('public-read')

In [71]:
generateheadlinefile('news_sources_au.json', 'data-au.json', "au", getheadlines)
generateheadlinefile('news_sources_us.json', 'data-us.json', 'us', getheadlines_JS)

[{'headline': 'All options on the table in wake of Syrian gas attack, US warns', 'sentiment': {'neg': 0.29, 'neu': 0.71, 'pos': 0.0, 'compound': -0.5423}, 'v_neg': [], 's_neg': [{'word': 'attack', 'score': -2.1}], 'v_pos': [], 's_pos': []}, {'headline': 'Father grieves for baby twins killed in Syrian gas massacre', 'sentiment': {'neg': 0.487, 'neu': 0.513, 'pos': 0.0, 'compound': -0.8225}, 'v_neg': [{'word': 'killed', 'score': -3.5}], 's_neg': [{'word': 'grieves', 'score': -2.1}], 'v_pos': [], 's_pos': []}, {'headline': "Rockhampton's flooded Fitzroy to peak at midday", 'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'v_neg': [], 's_neg': [], 'v_pos': [], 's_pos': []}, {'headline': 'Controversial Trump adviser Bannon dropped from National Security Council', 'sentiment': {'neg': 0.161, 'neu': 0.625, 'pos': 0.214, 'compound': 0.1531}, 'v_neg': [], 's_neg': [{'word': 'Controversial', 'score': -0.8}], 'v_pos': [], 's_pos': [{'word': 'Security', 'score': 1.4}]}, {'headli